# Example 2: Coupled Transmon Qubits

In [1]:
%load_ext autoreload
%autoreload 2
import time

In [2]:
import json
import design as d
import names as n

with open("design_variables.json") as in_file:
    initial_design_variables = json.load(in_file)

from qdesignoptimizer.utils.chip_generation import create_chip_base, ChipType
from qdesignoptimizer.utils.utils import close_ansys

In [3]:
# for this example to store the data in the right place
import os

os.makedirs(os.path.dirname("out/"), exist_ok=True)

## Design assembly

In [4]:
CHIP_NAME = "multi_transmon_chip"
OPEN_GUI = False
chip_type = ChipType(size_x="10mm", size_y="10mm", size_z="-300um")
design, gui = create_chip_base(
    chip_name=CHIP_NAME, chip_type=chip_type, open_gui=OPEN_GUI
)

n.add_design_variables_to_design(design, initial_design_variables)

In [5]:
def render_qiskit_metal_design(design, gui):
    d.add_transmon_plus_resonator(design, group=n.NBR_1)
    d.add_transmon_plus_resonator(design, group=n.NBR_2)

    d.add_coupler(design)

    d.add_route_interconnects(design)

    d.add_launch_pads(design)

    d.add_chargeline(design, group=n.NBR_1)
    d.add_chargeline(design, group=n.NBR_2)
    if gui:
        gui.rebuild()
        gui.autoscale()


render_qiskit_metal_design(design, gui)

## MiniStudies

In [6]:
import mini_studies as ms
import optimization_targets as ot
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qdesignoptimizer.design_analysis_types import MeshingMap

### Single resonator-qubit system
Useful when first tuning up subsystems

In [6]:
def CoupledLineTee_mesh_names(comp_names):
    all_names_to_mesh = [f"prime_cpw_{comp_names}", f"second_cpw_{comp_names}"]
    return all_names_to_mesh

In [7]:
MINI_STUDY_GROUP = n.NBR_1
MINI_STUDY = ms.get_mini_study_qb_res(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=[MINI_STUDY_GROUP],
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=True,
    opt_target_resonator_qubit_chi=True,
)

NameError: name 'ms' is not defined

In [ ]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS,
    meshing_map=[
        MeshingMap(component_class=CoupledLineTee, mesh_names=CoupledLineTee_mesh_names)
    ],
)

group_runs = 2  # 10
group_passes = 8  # 6
delta_f = 0.001
for i in range(group_runs):
    design_analysis.update_nbr_passes(group_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

In [ ]:
design_analysis.overwrite_parameters()

### Two qubit-resonator system

In [ ]:
MINI_STUDY = ms.get_mini_study_2qb_resonator_coupler()
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=[n.NBR_1, n.NBR_2],
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=False,
    opt_target_resonator_qubit_chi=True,
    opt_target_coupler_freq=True,
)

In [ ]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_TWO_QB,
)

group_runs = 2
group_passes = 6
delta_f = 0.001
for i in range(group_runs):
    design_analysis.update_nbr_passes(group_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

In [ ]:
design_analysis.overwrite_parameters()

### Purcell decay into charge line

The T1 of the qubit is limited by its decay into the charge line. This ministudy will change the distance of the charge line from the transmon pocket to optimize a T1 limit of 1 ms. This value is computed with simple capacitance matrix simulations and a classical model, assuming that the qubit frequency has already been optimized.

In [ ]:
close_ansys()

In [ ]:
MINI_STUDY_GROUP = 1
MINI_STUDY = ms.get_mini_study_qb_charge_line(group=MINI_STUDY_GROUP)
opt_targets = ot.get_opt_targets_qb_charge_line(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

In [ ]:
# %matplotlib inline
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_CHARGE_LINE_DECAY,
)

In [ ]:
group_runs = 5
group_passes_cap = 5

for i in range(group_runs):
    design_analysis.update_nbr_passes_capacitance_ministudies(group_passes_cap)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

### Optimize capacitance target

In [9]:
MINI_STUDY_BRANCH = 1
MINI_STUDY = ms.get_mini_study_resonator_capacitance(group=MINI_STUDY_BRANCH)
opt_targets = ot.get_opt_target_capacitance(group=MINI_STUDY_BRANCH)

RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)
# %matplotlib inline
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_CAPACITANCE,
)

[INFO|2025-03-07 13:51:33]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}


pyaedt INFO: No project is defined. Project Project165 exists and has been read.
pyaedt INFO: No consistent unique design is present. Inserting a new design.
pyaedt INFO: Added design 'HFSS_6YE' of type HFSS.
pyaedt INFO: Aedt Objects initialized
pyaedt INFO: Variable Manager initialized
pyaedt INFO: Design Loaded
pyaedt INFO: Successfully loaded project materials !
pyaedt INFO: Materials Loaded


[INFO|2025-03-07 13:51:34]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}
INFO 01:51PM [connect_project]: Connecting to Ansys Desktop API...
INFO 01:51PM [load_ansys_project]: 	Opened Ansys App
INFO 01:51PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 01:51PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/nirpit/Documents/Ansoft/
	Project:   Project165
INFO 01:51PM [connect_design]: 	Opened active design
	Design:    HFSS_6YE [Solution type: DrivenModal]
WARNING 01:51PM [connect_setup]: 	No design setup detected.
WARNING 01:51PM [connect_setup]: 	Creating driven modal default setup.
INFO 01:51PM [get_setup]: 	Opened setup `Setup`  (<class 'p

In [ ]:
group_runs = 10
group_passes_cap = 2

for i in range(group_runs):
    design_analysis.update_nbr_passes_capacitance_ministudies(group_passes_cap)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

### optimise scattering analysis and eigenmode

In [7]:
MINI_STUDY_BRANCH = 1
MINI_STUDY = ms.get_mini_study_scattering_analysis(group=MINI_STUDY_BRANCH)
opt_targets = ot.get_opt_target_scattering_analysis(group=MINI_STUDY_BRANCH)

RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)
# %matplotlib inline
design_analysis_state = DesignAnalysisState(
    
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_SCATTERING,
    
)

[INFO|2025-03-07 16:09:59]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}


pyaedt INFO: using existing logger.
pyaedt INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.
pyaedt INFO: AEDT installation Path C:\Program Files\AnsysEM\AnsysEM21.2\Win64.
pyaedt INFO: Launching AEDT with module PythonNET.
pyaedt INFO: Ansoft.ElectronicsDesktop.2021.2 Started with process ID 30180.
pyaedt INFO: pyaedt v0.6.46
pyaedt INFO: Python version 3.10.0 | packaged by conda-forge | (default, Nov 20 2021, 02:18:13) [MSC v.1916 64 bit (AMD64)]
pyaedt INFO: Project Project173 has been created.
pyaedt INFO: No design is present. Inserting a new design.
pyaedt INFO: Added design 'HFSS_5HS' of type HFSS.
pyaedt INFO: Aedt Objects initialized
pyaedt INFO: Variable Manager initialized
pyaedt INFO: Design Loaded
pyaedt INFO: Successfully loaded project materials !
pyaedt INFO: Materials Loaded


[INFO|2025-03-07 16:10:13]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}
INFO 04:10PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:10PM [load_ansys_project]: 	Opened Ansys App
INFO 04:10PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 04:10PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/nirpit/Documents/Ansoft/
	Project:   Project173
INFO 04:10PM [connect_design]: 	Opened active design
	Design:    HFSS_5HS [Solution type: DrivenModal]
WARNING 04:10PM [connect_setup]: 	No design setup detected.
WARNING 04:10PM [connect_setup]: 	Creating driven modal default setup.
INFO 04:10PM [get_setup]: 	Opened setup `Setup`  (<class 'p

In [ ]:
group_runs = 7
group_passes = 10
delta_f = 0.001
for i in range(group_runs):
    design_analysis.update_nbr_passes(group_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

[INFO|2025-03-07 16:10:14]: Updated_design_vars
{
    "cpw_width": "10 um",
    "cpw_gap": "6 um",
    "design_var_width_qubit_1": "400um",
    "design_var_width_qubit_2": "400um",
    "design_var_cl_pos_x_qubit_1": "-2600um",
    "design_var_cl_pos_y_qubit_1": "-1800um",
    "design_var_cl_pos_x_qubit_2": "-2600um",
    "design_var_cl_pos_y_qubit_2": "1800um",
    "design_var_length_resonator_1_capacitance": "20um",
    "design_var_length_resonator_2_capacitance": "20um",
    "design_var_lj_qubit_1": "12.1nH",
    "design_var_lj_qubit_2": "9.1nH",
    "design_var_cj_qubit_1": "0fF",
    "design_var_cj_qubit_2": "0fF",
    "design_var_length_resonator_1": "7500um",
    "design_var_length_resonator_2": "6500um",
    "design_var_length_coupler_1to2": "4000um",
    "design_var_coupl_length_qubit_1_resonator_1": "100um",
    "design_var_coupl_length_qubit_2_resonator_2": "100um",
    "design_var_coupl_length_resonator_1_tee": "170um",
    "design_var_coupl_length_resonator_2_tee": "160um"


pyaedt ERROR: Error in :   File "c:\Users\nirpit\AppData\Local\anaconda3\envs\qdesign-env\lib\site-packages\pyaedt\application\Design.py", line 2851, in delete_design
pyaedt ERROR: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024371), None) - delete_design -  GENERAL OR AEDT ERROR.
pyaedt ERROR: Project: Project173, Design: ('get_mini_study_scattering_analysis',) (Eigenmode), [error] Script macro error: Unable to locate Design: ('get_mini_study_scattering_analysis',)_drivenmodal (4:10:14 pm  Mar 07, 2025)

pyaedt ERROR: Arguments with values: 
pyaedt ERROR:     name = ('get_mini_study_scattering_analysis',)_drivenmodal 
pyaedt ERROR:     fallback_design = None 
pyaedt ERROR: Check Online documentation on: https://aedt.docs.pyansys.com/search.html?q=delete_design
pyaedt INFO: Project Project173 Saved correctly


[INFO|2025-03-07 16:10:17]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}
INFO 04:10PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:10PM [load_ansys_project]: 	Opened Ansys App
INFO 04:10PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 04:10PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/nirpit/Documents/Ansoft/
	Project:   Project173
INFO 04:10PM [connect_design]: 	Opened active design
	Design:    HFSS_5HS [Solution type: DrivenModal]
INFO 04:10PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)
INFO 04:10PM [connect]: 	Connected to project "Project173" and design "HFSS_5HS" 😀 

04:10PM 17s WARNING 

Design "('get_mini_study_scattering_analysis',)" info:
	# eigenmodes    1
	# variations    1
Design "('get_mini_study_scattering_analysis',)" info:
	# eigenmodes    1
	# variations    1


INFO 04:12PM [connect_design]: 	Opened active design
	Design:    ('get_mini_study_scattering_analysis',)_drivenmodal [Solution type: DrivenModal]
WARNING 04:12PM [connect_setup]: 	No design setup detected.
WARNING 04:12PM [connect_setup]: 	Creating driven modal default setup.
INFO 04:12PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)
INFO 04:13PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)
INFO 04:13PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)
INFO 04:13PM [analyze]: Analyzing setup Setup : Sweep


...

INFO 04:40PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)
INFO 04:40PM [analyze]: Analyzing setup Setup : Sweep1


pinfo_setup
solution_freq default 5.9239244645GHz
Kappa simualted is 56130.4887894596


[INFO|2025-03-07 16:41:26]: Updated_design_vars
{
    "design_var_coupl_length_resonator_1_tee": "500.0 um",
    "design_var_length_resonator_1": "4444.83794275589 um"
}


pyaedt INFO: Project Project173 Saved correctly


[INFO|2025-03-07 16:41:29]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}
INFO 04:41PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:41PM [load_ansys_project]: 	Opened Ansys App
INFO 04:41PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 04:41PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/nirpit/Documents/Ansoft/
	Project:   Project173
INFO 04:41PM [connect_design]: 	Opened active design
	Design:    HFSS_5HS [Solution type: DrivenModal]
INFO 04:41PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)
INFO 04:41PM [connect]: 	Connected to project "Project173" and design "HFSS_5HS" 😀 

04:41PM 30s WARNING 

## View Optimization results

In [ ]:
design_analysis.get_cross_kerr_matrix(iteration=-1)

In [14]:
design_analysis.get_eigenmode_results()

IndexError: list index out of range

In [ ]:
close_ansys()